In [292]:
from urllib.robotparser import RobotFileParser
from requests import request, get, post
from requests.compat import urljoin, urlparse, urlunparse , quote, unquote
from requests.exceptions import HTTPError
from time import sleep
from random import randint
from html import unescape
import re
from bs4 import BeautifulSoup
from string import punctuation
import json
import requests
from requests import Session
from requests import post
from requests.compat import urlencode
from requests import Session
from requests.cookies import cookiejar_from_dict
import pandas as pd
import time
import datetime
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [293]:
def contentcreate(key,sort):
    header={"user-agent" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    #헤더 필수

    url = 'https://search.naver.com/search.naver?where=news' #뉴스 검색 url
    

    resp = request('GET', url, params={'sm':'tab_jum','query':key,'sort':sort}) #sort 0이면 관련도 순 1이면 최신순
    

    news_search = BeautifulSoup(resp.text, 'html.parser') #텍스트를 html로 전환
    

    def linkto(number):  #링크 태그 생성 함수
        a='#sp_nws'
        b=str(number)
        c=' > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)'
        return a+b+c
   

    def titleto(number):  #타이틀 태그 생성 함수
        a='#sp_nws'
        b=str(number)
        c=' > div.news_wrap.api_ani_send > div > a'
        return a+b+c
    
     
    news_link2=[]  #빈리스트 생성
    title2=[]     #빈리스트 생성
    for count in range(1,13):
        linktag=linkto(count)
        titletag=titleto(count)
        link=news_search.select_one(linktag)
        title=news_search.select_one(titletag)
        if link:
            news_link2.append(link['href'])
            title2.append(title['title'])

#13번 돌려서 나온 기사 타이틀이랑 링크 리스트로 저장
    

 
    return news_link2, title2 #뉴스링크랑 제목 리스트 리턴


In [294]:
def titlecontent(news_link2):
    content5=[] #빈리스트 생성

    
    for i in range(0,len(news_link2)):    
        news = requests.get(news_link2[i],headers=header) #그 뉴스링크에 다시 접근
        
        news_html = BeautifulSoup(news.text,"html.parser") #html로 변환

        if news_link2[i].find('sports')>0:
            content=news_html.find_all('div', {'id':'newsEndContents'})  ##스포츠 일때는 sid가 없어서 1순위 확인
            content=str(content)
            content=content.split('<p class="source">') 
            content2=content[0]
        elif news_link2[i][-3:]=='106':
            content=news_html.find_all('div', {'id':'articeBody'})  ##엔터테이너 일때 sid = 106
            content2= str(content)
        else:
            content = news_html.select("div#dic_area")  ##일반뉴스일때 sid = 100~105 반례가 있을 수 있음 여기서
            content2= str(content)




        def remove_image(content): #이미지 설명 제거
            cleanr =re.compile('<em class="img_desc">.*?</em>')
            cleantext = re.sub(cleanr, '', content)
            return cleantext

        def remove_tag(content): #html태그 제거
            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', content)
            return cleantext

        def remove_email(content): #이메일 제거
            cleanr =re.compile('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)')
            cleantext = re.sub(cleanr, '', content)
            return cleantext

        content4=content2 #저장 혹시 모르니
        content2= remove_image(content2) #이미지 설명 제거
        content3= remove_email(content2)  #이메일 제거

        content3=BeautifulSoup(content3) #다시 html로 
        content3=content3.get_text() #텍스트만 추출 str
        content3 ##본문


        
        content5.append(content3) #본문을 5에 리스트로 어펜드

    return content5 #본문 리스트 리턴

In [280]:


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--lang=ko_KR')
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument('--window-size=1920x1080')

    # headless임을 숨기기 위해서
    # headless인 경우 Cloudflare 서비스가 동작한다.
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=chrome_options)
    return driver




In [299]:

def mainkey(key,sort='0'):
    news_link2,title2=contentcreate(key,sort) #함수 사용
    content5 = titlecontent(news_link2) #함수 사용
    con=" ".join(content5) #띄어쓰기 기준으로 뉴스기사 합체
    titlekeywords=con.split(' ') #띄어쓰기 기준으로 뉴스 키워드 분할

    if __name__ == "__main__":
        driver = set_chrome_driver()

        url = 'https://namu.wiki/w/'+key
        driver.get(url=url)
        print(driver.title)
        html = driver.page_source

        driver.close()


    soup = BeautifulSoup(html)
    imglink1=soup.select_one('tr:nth-child(2) > td > div > div > a > span > span > img.JGi8zLZN')



    if imglink1: #있으면 이미지 링크 할당하고 끝
        imglink=imglink1['src']
    else: #없으면...
        q1=soup.find_all('a', {'class':'x3PmqoJp'}) #이름 + 수식어 다 가져옴
        qqq=[] #이름 수식어 들어갈 리스트



        for i in range(0,int(len(q1))):
            if q1[i]['title'][0:3]==key: #이름으로 시작하는 이름 + 수식어만 가져옴
                qqq.append(q1[i]['title'][3:]) #수식어 부분만 다시 추출

        aaa = [0 for i in range(len(qqq))] #그만큼 빈 리스트

        for i in range(0,len(qqq)):
            if qqq[i] in titlekeywords: #위에서 띄어 쓰기 단위로 분할한 뉴스키워드에 수식어가 있으면 플러스 1
                aaa[i]=aaa[i]+1


        key2=qqq[aaa.index(max(aaa))] #가장 많은 빈도의 수식어 추출

        key3=key+key2 #키 3에 새로운 키를 할당


        if __name__ == "__main__": # 키 3로 다시돌림
            driver = set_chrome_driver()

            url = 'https://namu.wiki/w/'+key3
            driver.get(url=url)
            print(driver.title)
            html = driver.page_source

            driver.close()
            soup = BeautifulSoup(html)
            imglink1=soup.select_one('tr:nth-child(2) > td > div > div > a > span > span > img.JGi8zLZN')
            imglink=imglink1['src']




    imglink # 생성된 이미지 링크

    imglink2="https:"+imglink
    resp=requests.get(imglink2)
    filename=key+',png'
    with open(filename, 'wb') as f:
                          f.write(resp.content)  ##해당키워드 이름으로 png 파일로 저장
            
            
    return title2[0], content5[0]

In [304]:
key='한지민'
sort='0'# 0이면 관련도순 1이면 최신순
title,content=mainkey(key,sort)

한지민 - 나무위키
